## Minimal Prompt Change -> Result Change

This notebook contains an example of how a small, unrelated change in the prompt can change the result. This should be a warning against changing state prompts without testing!



### Non-Project imports/setup

In [1]:
import os
from pathlib import Path

# Set working dir to project root
root_dir_name = "the-curator"

if not Path.cwd().name == root_dir_name:
    for parent in Path.cwd().parents:
        if parent.name == root_dir_name:
            os.chdir(parent)
            print(f"Current working directory set to: {os.getcwd()}")
            break
    else:
        raise RuntimeError(f"The root directory '{root_dir_name}' is not a parent of the current working directory.")

import logging  # noqa E402
import os  # noqa E402
import sys  # noqa E402
import warnings  # noqa E402
from datetime import datetime  # noqa E402
from zoneinfo import ZoneInfo  # noqa E402

import pandas as pd  # noqa E402
from dotenv import load_dotenv  # noqa E402

# test that the current working dir is "the-curator"
if os.path.basename(os.path.abspath("..")) != "the-curator":
    raise RuntimeError("This script must be run from the 'the-curator' directory.")

# must happen before project imports!
if not (os.path.exists(".env") and os.path.exists(".env.db")):
    raise FileNotFoundError("No .env or .env.db found.")

load_dotenv(".env")
load_dotenv(".env.db")

logging.basicConfig(level=logging.WARN)

# Suppress FutureWarning from transformers
warnings.filterwarnings("ignore", category=FutureWarning)
logging.getLogger("transformers").setLevel(logging.ERROR)


Current working directory set to: /home/fab/Dokumente/Schule/Universität/12. Semester inkl Ilias/Info LLM Praktikum/the-curator


### Project imports/setup

In [2]:
from src.backend.modules.llm.lm_studio_llm import LMStudioLLM

task_llm = LMStudioLLM("meta-llama-3.1-8b-instruct", 0.05, 2048)

/home/fab/Dokumente/Schule/Universität/12. Semester inkl Ilias/Info LLM Praktikum/the-curator/.venv1/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
def send_to_llm(message, iterations):
    all = []
    for i in range(iterations):
        print(f"\r{i}   ", end="")
        res = task_llm.generate([{"role": "user", "content": message}])
        res = res.replace("    ", "  ")  # we are not here to compare spaces. That is not the point.
        all.append(res)

    if len(set(all)) == 1:
        print(f"\nAll {iterations} responses are the same!")
    else:
        print(f"\nNot all responses are the same! Got {len(set(all))} different responses.")
        print("\n---------------\n".join(set(all)))

    return all[0]

### Message comparison

In [4]:
message_1 = """You are an assistant of a flashcard management system.
You assist a user in executing tasks (creating/modifying/deleting cards/decks etc.).

The user gave the following input:

In the java_10 deck, please execute the following two tasks:
* Replace all mentions of 'method' by 'function'.
* Remove all mentions of 'in Java' etc.

Thanks!

You decided to search for cards. You wanted me to present you every single card you found.
I will now show you the cards one-by-one. You will only ever be able to see a single card. Here is the current card:

Card  from the deck Java Programming 10 with flag none and state new.

**Question**: What is the purpose of the 'main' method in Java?

**Answer**: It's the entry point to start program execution.

You have the following options:

 * Doing nothing: Respond "do_nothing".
 * Delete that card: Respond "delete_card".
 * Edit that card. Respond with "edit_card" and the following, filled-out template:
 {
    "question": "<new question here>",
    "answer": "<new answer here>",
    "flag": "<new flag here>",
    "state": "<new card state here>"
 }
  These flag options exist: ["none", "red", "orange", "green", "blue", "pink", "turquoise", "purple"]
  These card state options exist: ["new", "learning", "review", "suspended", "buried"]
  If you do not wish to change some attributes, you should omit it from your response.

Please answer only with the operation you want to perform in the given format, and answer nothing else!
""".strip()

In [5]:
message_2 = message_1.replace("'method'", "'methods'")

In [6]:
res_1 = send_to_llm(message_1, 10)
print(res_1)

9   
All 10 responses are the same!
edit_card {
  "question": "What is the purpose of the 'main' function in Java?",
  "answer": "It's the entry point to start program execution.",
  "flag": "none",
  "state": "new"
}


In [7]:
res_2 = send_to_llm(message_2, 10)
print(res_2)

9   
All 10 responses are the same!
edit_card {
  "question": "What is the purpose of the 'main' function?",
  "answer": "It's the entry point to start program execution.",
  "flag": "none",
  "state": "new"
}


### Backup: Original code creating prompt

In [8]:
# # original code
# tests = load_test_data("tests/data/tests.json")
# _prompt_template = """
# You are an assistant of a flashcard management system.
# You assist a user in executing tasks (creating/modifying/deleting cards/decks etc.).
#
# The user gave the following input:
#
# {user_input}
#
# You decided to search for cards. You wanted me to present you every single card you found.
# I will now show you the cards one-by-one. You will only ever be able to see a single card. Here is the current card:
#
# {card}
#
# You have the following options:
#
#  * Doing nothing: Respond "do_nothing".
#  * Delete that card: Respond "delete_card".
#  * Edit that card. Respond with "edit_card" and the following, filled-out template:
#  {{
#     "question": "<new question here>",
#     "answer": "<new answer here>",
#     "flag": "<new flag here>",
#     "state": "<new card state here>"
#  }}
#   These flag options exist: ["none", "red", "orange", "green", "blue", "pink", "turquoise", "purple"]
#   These card state options exist: ["new", "learning", "review", "suspended", "buried"]
#   If you do not wish to change some attributes, you should omit it from your response.
#
# Please answer only with the operation you want to perform in the given format, and answer nothing else!
# """.strip()
# prompt = "In the java_10 deck, please execute the following two tasks:\n* Replace all mentions of 'method' by 'function'.\n* Remove all mentions of 'in Java' etc.\n\nThanks!"
# cards = tests.test_flashcard_manager.get_deck_by_name("Java Programming 10").cards
#
# card = cards[0]
# card.question
